In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [12]:
%pip install langchain -q
%pip install ibm-watson-machine-learning -q
%pip install wget -q
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install singlestoredb -q
%pip install sqlalchemy-singlestoredb -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
import shutil
import argparse

In [3]:
try:
    wxa_url = os.getenv("WATSONX_URL")
except KeyError:
    wxa_url = getpass.getpass("Please enter your watsonx.ai URL domain (hit enter): ")

In [4]:
try:
    wxa_api_key = os.getenv("WXA_API_KEY")
except KeyError:
    wxa_api_key = getpass.getpass("Please enter your watsonx.ai API key (hit enter): ")

In [5]:
try:
    wxa_project_id = os.getenv("WXA_PROJECT_ID")
except KeyError:
    wxa_project_id = getpass.getpass("Please enter your watsonx.ai Project ID (hit enter): ")

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def get_embedding_function():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.vectorstores import Chroma

In [8]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [9]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [10]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [11]:
def calculate_chunk_ids(chunks):

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

In [12]:
def add_to_chroma(chunks: list[Document]):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = db.get(include=[]) 
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [13]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


In [14]:
def main():
    documents = load_documents()
    chunks = split_documents(documents)
    add_to_chroma(chunks)

In [15]:
if __name__ == "__main__":
    main()

Ignoring wrong pointing object 302 0 (offset 0)
Ignoring wrong pointing object 368 0 (offset 0)
/var/folders/r5/t2d_mp_s33l94mrmkvvd3xvr0000gn/T/ipykernel_53612/573115512.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/var/folders/r5/t2d_mp_s33l94mrmkvvd3xvr0000gn/T/ipykernel_53612/823623245.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma imp

Number of existing documents in DB: 175
✅ No new documents to add


In [16]:
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

In [17]:
PROMPT_TEMPLATE="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

In [19]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

In [20]:
credentials = {
    "url": "https://#########m.com",
    "apikey": "################-GcFeLxv7CzUU_wya"
}

In [21]:
# model = ModelTypes.GRANITE_13B_CHAT_V2

model = Model(
    model_id=ModelTypes.GRANITE_13B_CHAT_V2,
    params=parameters,
    credentials={
        "url": credentials.get("url"),
        "apikey": credentials.get("apikey")
    },
    project_id="############-2c6c8c483cc3"
)
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
granite_llm_ibm = WatsonxLLM(model=model)
# from langchain.llms import WatsonxLLM

# granite_llm_ibm = WatsonxLLM(model_id=model,
#     url=os.getenv("WATSONX_URL"),
#     apikey=os.getenv("WXA_API_KEY"),
#     project_id=os.getenv("WXA_PROJECT_ID"),
#     params=parameters)

In [22]:
def query_rag(query_text: str):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())
    
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)
    response_text = granite_llm_ibm.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [24]:

# query_text = getpass.getpass("Please Enter you query here!")
query_text = input("Please Enter your query here: ")
query_rag(query_text)

Response: Acne is a common skin condition that occurs when hair follicles become clogged with oil and dead skin cells. This can result in whiteheads, blackheads, or pimples.
Sources: ['data/Antibioitcs_guide_2013.pdf:15:2', 'data/Antibioitcs_guide_2013.pdf:16:3', 'data/Antibioitcs_guide_2013.pdf:29:1', 'data/Antibioitcs_guide_2013.pdf:14:2', 'data/Antibioitcs_guide_2013.pdf:13:2']


'Acne is a common skin condition that occurs when hair follicles become clogged with oil and dead skin cells. This can result in whiteheads, blackheads, or pimples.'